In [1]:
import pandas as pd

#sun_df = pd.read_csv('copy_number/hg38_features.txt', sep='\t')
#sun_df.columns = ['loc', 'NAB', 'NC', 'ND', 'NN']
#sun_df = (sun_df.set_index('loc') * 2).reset_index()
#sun_df.head()

In [2]:


# try using the SUN VCF
import vcf
vcf_path = "/hive/users/ifiddes/amplicon_analysis_with_new_suns/all_samples_reference_free/index/Notch2NL_SUN_UniqueIndels_ConsensusRef.vcf.gz"

vcf_recs = list(vcf.Reader(open(vcf_path)))

sun_positions = []
for vcf_rec in vcf_recs:
    if vcf_rec.is_indel:
        continue
    sun_positions.append([vcf_rec.POS - 1] + map(int, [x.gt_alleles[0] for x in vcf_rec.samples]))
sun_df = pd.DataFrame(sun_positions, columns=['loc', 'NN', 'NA', 'NB', 'NC', 'ND'])
sun_df = (sun_df.set_index('loc') * 2).reset_index()

In [3]:
# load SUN counts
sun_counts = [x.split() for x in open('/hive/users/cbosworth/imputation/4222.txt')]
m = {}
for x in sun_counts:
    n = os.path.basename(x[0]).split('.')[0]
    ab = int(x[1][-1]) + int(x[2][-1])
    c = int(x[3][-1])
    d = int(x[4][-1])
    m[n] = [4, c, d, 2]

In [4]:
df = pd.read_csv('NA19240.pileup.txt', sep='\t', index_col=0)

df.head()

,loc,ref,A,C,G,T,coverage,ref_count,alt_count,ratio
0,0,T,0,0,0,1079,1079,1079,0,0.000000
1,1,G,5,0,1081,1,1087,1081,6,0.005520
2,2,G,0,0,1358,1,1359,1358,1,0.000736
3,3,G,0,1,1385,2,1388,1385,3,0.002161
4,4,G,0,1,1416,1,1418,1416,2,0.001410


In [5]:
def calculate_deviance(s):
    if s.alt_count == 0 or s.coverage <= 20:
        return None
    deviance = (s.expected_alt - s.adjusted_alt) / (np.sqrt(1.0 * s.coverage * s.ratio * (1 - s.ratio)))
    return deviance

def calculate_expected(s, tot):
    """Expected value"""
    expected_ratio = (1.0 * s.NA + s.NB + s.NC + s.ND + s.NN) / tot
    return expected_ratio * s.coverage

In [7]:
import numpy as np
tot = 10
df_m = df.merge(sun_df, on='loc')
df_m['adjusted_alt'] = [x * (1.0 * tot / 10) for x in df_m['alt_count']]
df_m['expected_alt'] = df_m.apply(calculate_expected, axis=1, tot=tot)
df_m['deviance'] = df_m.apply(calculate_deviance, axis=1)

In [8]:
df_m.head()

,loc,ref,A,C,G,T,coverage,ref_count,alt_count,ratio,NN,NA,NB,NC,ND,adjusted_alt,expected_alt,deviance
0,1799,A,2314,5,467,5,2791,2314,477,0.170906,2,0,0,0,0,477.0,558.2,4.083148
1,1921,C,461,2755,8,5,3229,2755,474,0.146795,0,0,0,0,2,474.0,645.8,8.542937
2,1983,G,500,10,2334,10,2854,2334,520,0.182200,2,0,0,0,0,520.0,570.8,2.463421
3,2136,C,8,2000,6,133,2147,2000,147,0.068468,0,2,0,0,0,147.0,429.4,24.132763
4,2175,T,10,413,13,1805,2241,1805,436,0.194556,2,0,0,0,0,436.0,448.2,0.651027


In [10]:
from tools.bio import *
_, seq = read_fasta('/hive/users/cbosworth/refs/notch/notch2_aligned_consensus.fasta', None).next()

In [16]:
121.0/(634+121)

0.16026490066225166

In [ ]:
from collections import *
deviances = defaultdict(list)
for n, df in dev_dfs.iteritems():
    for p, d in zip(df['loc'], df['deviance']):
        deviances[p].append(d)

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
vals = sorted(deviances.iteritems(), key=lambda x: x[0])
positions, vals = zip(*vals)
vals = [np.nanmean(x) for x in vals]
plt.plot(positions, vals)

In [ ]:
# how many sites are always ratio of 0?
site_map = defaultdict(list)
for n, df in dev_dfs.iteritems():
    for p, d in zip(df['loc'], df['ratio']):
        site_map[p].append(d)

bad_sites = set()
for s, x in site_map.iteritems():
    if all(v==0 for v in x):
        bad_sites.add(s)

In [ ]:
print len(bad_sites)

In [ ]:
len(positions)

In [9]:
# need to plot this per-paralog
os.getcwd()

'/hive/users/ifiddes/notch2nl_berkeley_data/imputation_pipeline'

In [22]:
seq[1606-15:1606+16]

'TTCACATTGTGTTTTGCTTGGTTTTATATAT'

In [21]:
len('TTCACATTGTGTTTTGCTTGGTTTTATATAT')

30

In [ ]:
TTCACATTGTGTTTTTCTTGGTTTTATATAT